# vaxxAId

**VaxxAId** aims to cluster and find similarities among patients who experienced COVID-19 vaccine adverse reactions or side effects based on the reports from the Vaccine Adverse Event Reporting System (VAERS). This would aid in the identification and classification of patients at risk of such effects. It will use the **K-modes clustering algorithm** to cluster the reports based on features such as patient outcomes, demographics, type of vaccine received, etc.

This project is developed by:
- Rene Lee Aquitania
- Alyssa Beatrice Diokno
- Derryn Joi Martirez
- Marie Isabel Yao

## PART 1: Data Preprocessing

### PART 1.1: Importing the Libraries

Before running the code below, make sure to install the following libraries using `conda install <package-name>`: 
- `pandas`
- `numpy`
- `matplotlib`
- `seaborn`
- `sklearn`

Install the following libraries as well using `pip install <package-name>`:
- `kmodes`
- `kneed`

In [2]:
# Importing libraries

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
# import ipynb.fs.defs.utils as utils

from kmodes.kmodes import KModes
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from kneed import KneeLocator

ModuleNotFoundError: No module named 'kneed'

### PART 1.2: Data Cleaning

We start by reading the following CSV files into separate dataframes. For each year, we have patient data and vaccine data from two dataframes that can be joined on the key `VAERS_ID`.

In [ ]:
patients2021 = pd.read_csv('../data/2021VAERSDATA.csv', encoding='latin1')
vaccine2021 = pd.read_csv('../data/2021VAERSVAX.csv', encoding='latin1')

patients2022 = pd.read_csv('../data/2022VAERSDATA.csv', encoding='latin1')
vaccine2022 = pd.read_csv('../data/2022VAERSVAX.csv', encoding='latin1')

# Joining the dataframes on VAERS_ID
patients2021 = patients2021.join(vaccine2021.set_index('VAERS_ID'), on='VAERS_ID', how='inner')
patients2022 = patients2022.join(vaccine2022.set_index('VAERS_ID'), on='VAERS_ID', how='inner')

Since the datasets may contain reports on adverse effects of other FDA-approved vaccines, we only keep rows where the vaccine type is COVID-19. Finally, we concatenate the 2021 and 2022 dataframes.

In [ ]:
patients2021 = patients2021[patients2021['VAX_TYPE']=='COVID19']
patients2022 = patients2022[patients2022['VAX_TYPE']=='COVID19']

patients = pd.concat([patients2021, patients2022], axis=0)

features = [
    'VAERS_ID','AGE_YRS','SEX','DIED','L_THREAT',
    'ER_VISIT','HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD',
    'V_ADMINBY','VAX_TYPE','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]
    
patients = patients[features]

patients = patients.drop(['VAX_TYPE'],axis=1)

patients.to_csv('../data/20212022DATA.csv', index=False)

patients_df = pd.read_csv('../data/20212022DATA.csv', encoding='latin1')

patients

Described in the following table are the attributes for the clustering process. All of these are categorical data.

| Attributes | Data Type | Description |
| ---------- | --------- | ----------- |
| `AGE_GRP`      | range     | Age group                                                   |
| `SEX`          | char(1)   | Sex                                                         |
| `DIED`         | char(1)   | Adverse effect caused death of the patient                  |
| `L_THREAT`     | char(1)   | Life-threatening event associated with the vaccination      |
| `ER_VISIT`     | char(1)   | Patient required ER visit after experiencing adverse effect |
| `HOSPITAL`     | char(1)   | Patient was hospitalized after experiencing adverse effect  |
| `DISABLE`      | char(1)   | Patient was disabled after experiencing adverse effect      |
| `BIRTH_DEFECT` | char(1)   | Patient has birth defect                                    |
| `RECOVD`       | char(1)   | Patient recovered from adverse effect                       |
| `V_ADMINBY`    | char(3)   | Vaccine administered at                                     |
| `VAX_MANU`     | char(40)  | Vaccine manufacterer                                        |
| `VAX_SITE`     | char(6)   | Vaccination anatomic site                                   |
| `VAX_ROUTE`    | char(6)   | Vaccine route of administration                             |

The following function further cleans the data in the specified columns as some null values are equal to `"N"` (for the yes or no attributes):
- `AGE_GRP`: maps `AGE_YRS` into an age group
- `DIED`: replaces null values with `N`
- `L_THREAT`: replaces null values with `N`
- `ER_VISIT`: replaces null values with `N`
- `HOSPITAL`: replaces null values with `N`
- `DISABLE`: replaces null values with `N`
- `BIRTH_DEFECT`: replaces null values with `N`

After passing the dataframe into the `replace_values` function, the only null values left in the dataframe are actual null values which we can drop.

In [ ]:
def replace_values(df):
    # Add a new column, AGE_GRP, and drop AGE_YRS
    age_max = df['AGE_YRS'].max()
    multiplier = 10 ** -1
    agegrp_max = int(math.ceil(age_max * multiplier) / multiplier)
    age_grp = [i for i in range(0,agegrp_max+1,10)]
    df['AGE_GRP'] = pd.cut(x=df['AGE_YRS'], bins=age_grp).astype(str)
    df = df.drop(['AGE_YRS'],axis=1)

    # Fill na with the following corresponding values
    values = {'DIED': 'N', 'L_THREAT': 'N', 'ER_VISIT': 'N', 'HOSPITAL':'N', 'DISABLE':'N', 'BIRTH_DEFECT':'N'}
    df = df.fillna(value=values)
    
    return df

patients = replace_values(patients)
patients

In [ ]:
# Checking the number of null values

patients.replace(["U","UN","UNK","UNKNOWN MANUFACTURER","nan"], np.nan, inplace=True)
patients.isna().sum()

In the succeeding code blocks, we drop the rows with null values. We also replace `"U"`, `"UNK"` and `"UNKNOWN MANUFACTURER"` values in the dataframe with `np.nan` for easy dropping.

In [ ]:
patients.dropna(inplace=True)
patients.isna().sum()

In [ ]:
# Remove rows where AGE_GRP is unknown

print(patients.AGE_GRP.value_counts())
print("\n")
print(patients.SEX.value_counts())
print("\n")
print(patients.RECOVD.value_counts())

In [ ]:
print(patients.V_ADMINBY.value_counts())
print("\n")
print(patients.VAX_MANU.value_counts())

In [ ]:
print(patients.VAX_SITE.value_counts())
print("\n")
print(patients.VAX_ROUTE.value_counts())

In [ ]:
# We can select the first 30000 rows only.

patients.drop('VAERS_ID', inplace=True, axis=1)

### PART 1.3: Data Transformation

We now map nominal features into numerical labels for 

In [ ]:
categorical = ['SEX','V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE']

le = LabelEncoder()

for i in categorical:
    patients[i] = le.fit_transform(patients[i])


In [ ]:
age_groups = {
    '(0, 10]': 0,
    '(10, 20]': 1,
    '(20, 30]': 2,
    '(30, 40]': 3,
    '(40, 50]': 4,
    '(50, 60]': 5,
    '(60, 70]': 6,
    '(70, 80]': 7,
    '(80, 90]': 8,
    '(90, 100]': 9,
    '(100, 110]': 10
}

patients.replace({'AGE_GRP': age_groups}, inplace=True)

In [ ]:
binary = ['DIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'DISABLE', 'BIRTH_DEFECT', 'RECOVD']

for i in binary:
    patients[i] = patients[i].apply(lambda x: 1 if x=="Y" else 0)

patients.head()

In [ ]:
columns = [
    'SEX', 'DIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL', 'DISABLE', 'BIRTH_DEFECT',
    'RECOVD', 'V_ADMINBY', 'VAX_MANU', 'VAX_SITE', 'VAX_ROUTE', 'AGE_GRP'
    ]

# sample = patients.groupby(columns).filter(lambda x: len(x) > 20)
sample = patients.head(10000)
# sample.to_csv('../data/finaldataset.csv', index=False)
sample


In [ ]:
sample.value_counts()

## PART 2: Exploratory Data Analysis

### PART 2.1: Univariate Analysis

We now create charts and visualize data for single features to further understand the dataset. The following graphs are for the **single features** found in the data set.

**SEX**

In [ ]:
sex_barplot = sns.catplot(x='SEX', kind='count', palette='Set2', data=sample)
sex_barplot.axes[0,0].set_xticks([0,1])
sex_barplot.axes[0,0].set_xticklabels(['Female', 'Male'])
sex_barplot.set(title='Sex of COVID-19 Patients')
sex_barplot.set_xlabels('Sex', clear_inner=False)
sex_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that there are **more female** patients in the data set than male patients and that the number of male patients in the data set is **less than half** the number of female patients

**DEATH COUNT**

In [ ]:
died_barplot = sns.catplot(x='DIED', kind='count', palette='Set2', data=sample)
died_barplot.axes[0,0].set_xticks([0,1])
died_barplot.axes[0,0].set_xticklabels(['Lived', 'Died'])
died_barplot.set(title='Living Status of Patients')
died_barplot.set_xlabels('Status', clear_inner=False)
died_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that more patients in the data set **lived** than died and that the difference is more than half.

**Age Groups**

In [ ]:
age_barplot = sns.catplot(x='AGE_GRP', kind='count', palette='magma', data=sample)
age_barplot.axes[0,0].set_xticks([0,1,2,3,4,5,6,7,8,9,10])
age_barplot.axes[0,0].set_xticklabels(['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'], rotation=45)
age_barplot.set(title='Age Groups of Patients')
age_barplot.set_xlabels('Age Groups', clear_inner=False)
age_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that most of the patients in the data set are aged **50 to 60 years old**. In general, majority of the patients in the data set are of the middle-aged to early senior group, specifically ages 30 to 60. Both extremes, the youngest age groups and the oldest age groups, make up less of the data set compared to the age groups in the middle.

**Life Threatening Event Associated with the Vaccination**

In [ ]:
lte_barplot = sns.catplot(x='L_THREAT', kind='count', palette='Set2', data=sample)
lte_barplot.axes[0,0].set_xticks([0,1])
lte_barplot.axes[0,0].set_xticklabels(['Without Event', 'With Event'])
lte_barplot.set(title='Patients with Life Threatening Events Associated with COVID-19 Vaccination')
lte_barplot.set_xlabels('Life Threatening Event', clear_inner=False)
lte_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that more patients in the data set **had no life-threatening events associated with the vaccination** compared to patients that had life-threatening events associated with the vaccination.

**Emergency Room Visits**

In [ ]:
er_barplot = sns.catplot(x='ER_VISIT', kind='count', palette='Set2', data=sample)
er_barplot.axes[0,0].set_xticks([0,1])
er_barplot.axes[0,0].set_xticklabels(['No ER Visit', 'With ER Visit'])
er_barplot.set(title='Patients with Emergency Room Visits Associated with the COVID-19 Vaccine')
er_barplot.set_xlabels('Emergency Room Visits', clear_inner=False)
er_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that there are **no patients in the data set** who visited the emergency room after experiencing adverse effects from the vaccine.

**Hospitalized**

In [ ]:
hosp_barplot = sns.catplot(x='HOSPITAL', kind='count', palette='Set2', data=sample)
hosp_barplot.axes[0,0].set_xticks([0,1])
hosp_barplot.axes[0,0].set_xticklabels(['Not Hospitalized', 'Hospitalized'])
hosp_barplot.set(title='Number of Hospitalized Patients Due to COVID-19 Vaccine')
hosp_barplot.set_xlabels('Number of Patients', clear_inner=False)
hosp_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that the number of patients present in the data set that were hospitalized after experiencing adverse effects is **less than half** the number of patients not hospitalized.

**Disabled**

In [ ]:
dis_barplot = sns.catplot(x='DISABLE', kind='count', palette='Set2', data=sample)
dis_barplot.axes[0,0].set_xticks([0,1])
dis_barplot.axes[0,0].set_xticklabels(['Not Disabled', 'Disabled'])
dis_barplot.set(title='Number of Patients Disabled due to COVID-19 Vaccine')
dis_barplot.set_xlabels('Number of Patients', clear_inner=False)
dis_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that the number of patients in the data set disabled after experiencing adverse effects is **less than a quarter** of the number of patients not disabled.

**Birth Defect**

In [ ]:
bd_barplot = sns.catplot(x='BIRTH_DEFECT', kind='count', palette='Set2', data=sample)
bd_barplot.axes[0,0].set_xticks([0,1])
bd_barplot.axes[0,0].set_xticklabels(['With No Birth Defect', 'With Birth Defect'])
bd_barplot.set(title='Number of Patients with Birth Defect')
bd_barplot.set_xlabels('Number of Patients', clear_inner=False)
bd_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that the number of patients in the data set with a birth defect is **zero**.

**Recovered**

In [ ]:
rec_barplot = sns.catplot(x='RECOVD', kind='count', palette='Set2', data=sample)
rec_barplot.axes[0,0].set_xticks([0,1])
rec_barplot.axes[0,0].set_xticklabels(['Recovered', 'Did Not Recover'])
rec_barplot.set(title='Number of Patients that Recovered from Adverse Effects')
rec_barplot.set_xlabels('Number of Patients', clear_inner=False)
rec_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that the **number of patients in the data set who did not recover from adverse effects** is greater than the number of patients who recovered.

**Vaccine Administered At**

In [ ]:
loc_barplot = sns.catplot(x='V_ADMINBY', kind='count', palette='Set2', data=sample)
loc_barplot.axes[0,0].set_xticks([0,1,2,3,4,5,6,7])
loc_barplot.axes[0,0].set_xticklabels(['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'], rotation=45)
loc_barplot.set(title='Locations where COVID-19 Vaccine was Administered')
loc_barplot.set_xlabels('Locations', clear_inner=False)
loc_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that most of the vaccines were administered in **private**, and the places with the least amount of vaccines administered at are at school, in a pharmacy, and in the military.

**Vaccine Manufacturers**

In [ ]:
manu_barplot = sns.catplot(x='VAX_MANU', kind='count', palette='Set2', data=sample)
manu_barplot.axes[0,0].set_xticks([0,1,2])
manu_barplot.axes[0,0].set_xticklabels(['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'])
manu_barplot.set(title='COVID-19 Vaccine Manufacturers')
manu_barplot.set_xlabels('Vaccine Manufacturers', clear_inner=False)
manu_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that most patients in the data set have vaccines manufactured by **Janssen**. The number of patients with a Moderna vaccine is second, and the number of patients with a Pfizer/Biotech vaccine is zero.

**Vaccination Anatomic Site**

In [ ]:
ana_barplot = sns.catplot(x='VAX_SITE', kind='count', palette='Set2', data=sample)
ana_barplot.axes[0,0].set_xticks([0,1,2,3,4,5,6,7])
ana_barplot.axes[0,0].set_xticklabels(['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'])
ana_barplot.set(title='COVID-19 Vaccination Anatomic Sites')
ana_barplot.set_xlabels('Vaccination Anatomic Sites', clear_inner=False)
ana_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that the anatomic sites with the most number of patients vaccinated at are **LA, RA, and AR**.

**Route of Administration of Vaccine**

In [ ]:
route_barplot = sns.catplot(x='VAX_ROUTE', kind='count', palette='Set2', data=sample)
route_barplot.axes[0,0].set_xticks([0,1,2,4,5,3])
route_barplot.axes[0,0].set_xticklabels(['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other'], rotation=90)
route_barplot.set(title='Route of Administration of COVID-19 Vaccines')
route_barplot.set_xlabels('Routes of Administration of Vaccine', clear_inner=False)
route_barplot.set_ylabels('', clear_inner=False)

From the graph above, we can see that most patients in the data set had their vaccine administrated **intramuscularly**.

### PART 2.2: Bivariate Analysis

The following graphs are for analysis of **two features** found in the data set.

**Vaccine Manufacturers**

In [ ]:
manu_threat = sns.catplot(x='VAX_MANU', hue='L_THREAT', kind='count', data=sample)
#manu_threat.axes[0,0].set_xticks([0,1,2,4,5,3])
#manu_threat.axes[0,0].set_xticklabels(['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other'], rotation=90)
manu_threat.set_titles('Route of Administration of COVID-19 Vaccines')
manu_threat.set_xlabels('Routes of Administration of Vaccine', clear_inner=False)
manu_threat.set_ylabels('', clear_inner=False)

## PART 3: Model Implementation

### PART 3.1: Optimization of number of clusters

We used the **Elbow Method** to determine the optimal number of clusters into which the data might be clustered. We select the value of `k` at the "elbow" or the point after which the distortion/inertia starts decreasing in a linear fashion.

In [ ]:
cost = []
scores = []
K = range(2,16)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "Cao", n_init = 1, verbose=1)
    kmode.fit_predict(sample)
    cost.append(kmode.cost_)

    scores.append(metrics.silhouette_score(sample,kmode.labels_))
    
plt.plot(K, cost, 'bx-')
plt.xlabel('K clusters')
plt.ylabel('Cost')
plt.title('Elbow Method For Optimal k')
plt.show()

plt.plot(K,scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

Using the `KneeLocator` method from the `kneed` library, we identified the optimal number of clusters: $k=5$.

In [ ]:
optimal_k = KneeLocator(list(K), cost, curve='convex', direction='decreasing')

k = optimal_k.knee

k

### PART 3.2: K-Modes Clustering

We now proceed to fitting the model to the `patients` dataframe, setting the number of clusters to `n_clusters = 7` and using the Cao initialization for categorical data clustering. We then put the generated labels or cluster numbers in a new column `CLUSTER`.

In [ ]:
kmode = KModes(n_clusters=k, init = "Cao", n_init = 1, verbose=1)
labels = kmode.fit_predict(sample)
sample['CLUSTER'] = labels

sample

## PART 4: Results and Discussion

lagay dito observations as a whole, e.g. cluster 0 has the most number hospitalized patients, etc.

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

for i in features:
    plt.subplots(figsize = (15,5))
    sns.countplot(x='CLUSTER',hue=i, data = sample)
    plt.show()

**Identification of Each Cluster**

Cluster 0

In [4]:
cluster_0 = sample[sample['CLUSTER']==0].value_counts()
cluster_0.to_csv("../results/cluster_0.csv")

print(cluster_0.head(15))

NameError: name 'sample' is not defined

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

xtickslabels = [
    ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'],
    ['Female', 'Male'],
    ['Lived', 'Died'],
    ['Without Event', 'With Event'],
    ['No ER Visit', 'With ER Visit'],
    ['Not Hospitalized', 'Hospitalized'],
    ['Not Disabled', 'Disabled'],
    ['With No Birth Defect', 'With Birth Defect'],
    ['Recovered', 'Did Not Recover'],
    ['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'],
    ['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'],
    ['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'],
    ['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other']
]

In [3]:
for i in range(len(features)):
    plot_0 = sns.catplot(x=features[i], kind='count', palette='Set2', data=sample.loc[sample['CLUSTER']==0])
    plot_0.axes[0,0].set_xticks([x for x in range(0,len(xtickslabels[i]))])
    plot_0.axes[0,0].set_xticklabels(xtickslabels[i], rotation=90)
    #plot_0.set_xlabels('Routes of Administration of Vaccine', clear_inner=False)
    #plot_0.set_ylabels('', clear_inner=False)
    

NameError: name 'sample' is not defined

The following observations can be seen on patients for Cluster 0:
- aged 30 to 40
- female
- living
- without a life threatening event
- without an emergency room visit
- not hospitalized
- not disabled
- with no birth defect
- did not recover
- vaccinated in private
- vaccinated with Janssen
- vaccinated in LA site
- vaccinated intramuscularly

Cluster 1

In [ ]:
cluster_1 = sample[sample['CLUSTER']==1].value_counts()
cluster_1.to_csv("../results/cluster_1.csv")

print(cluster_1.head(15))

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

xtickslabels = [
    ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'],
    ['Female', 'Male'],
    ['Lived', 'Died'],
    ['Without Event', 'With Event'],
    ['No ER Visit', 'With ER Visit'],
    ['Not Hospitalized', 'Hospitalized'],
    ['Not Disabled', 'Disabled'],
    ['With No Birth Defect', 'With Birth Defect'],
    ['Recovered', 'Did Not Recover'],
    ['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'],
    ['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'],
    ['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'],
    ['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other']
]

for i in range(len(features)):
    plot_1 = sns.catplot(x=features[i], kind='count', palette='Set2', data=sample.loc[sample['CLUSTER']==1])
    plot_1.axes[0,0].set_xticks([x for x in range(0,len(xtickslabels[i]))])
    plot_1.axes[0,0].set_xticklabels(xtickslabels[i], rotation=90)

The following observations can be seen on patients for Cluster 1:


Cluster 2

In [ ]:
cluster_2 = sample[sample['CLUSTER']==2].value_counts()
cluster_2.to_csv("../results/cluster_2.csv")

print(cluster_2.head(15))

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

xtickslabels = [
    ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'],
    ['Female', 'Male'],
    ['Lived', 'Died'],
    ['Without Event', 'With Event'],
    ['No ER Visit', 'With ER Visit'],
    ['Not Hospitalized', 'Hospitalized'],
    ['Not Disabled', 'Disabled'],
    ['With No Birth Defect', 'With Birth Defect'],
    ['Recovered', 'Did Not Recover'],
    ['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'],
    ['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'],
    ['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'],
    ['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other']
]

for i in range(len(features)):
    plot_2 = sns.catplot(x=features[i], kind='count', palette='Set2', data=sample.loc[sample['CLUSTER']==2])
    plot_2.axes[0,0].set_xticks([x for x in range(0,len(xtickslabels[i]))])
    plot_2.axes[0,0].set_xticklabels(xtickslabels[i], rotation=90)

The following observations can be seen on patients for Cluster 2:

Cluster 3

In [ ]:
cluster_3 = sample[sample['CLUSTER']==3].value_counts()
cluster_3.to_csv("../results/cluster_3.csv")

print(cluster_3.head(15))

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

xtickslabels = [
    ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'],
    ['Female', 'Male'],
    ['Lived', 'Died'],
    ['Without Event', 'With Event'],
    ['No ER Visit', 'With ER Visit'],
    ['Not Hospitalized', 'Hospitalized'],
    ['Not Disabled', 'Disabled'],
    ['With No Birth Defect', 'With Birth Defect'],
    ['Recovered', 'Did Not Recover'],
    ['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'],
    ['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'],
    ['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'],
    ['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other']
]

for i in range(len(features)):
    plot_3 = sns.catplot(x=features[i], kind='count', palette='Set2', data=sample.loc[sample['CLUSTER']==3])
    plot_3.axes[0,0].set_xticks([x for x in range(0,len(xtickslabels[i]))])
    plot_3.axes[0,0].set_xticklabels(xtickslabels[i], rotation=90)

The following observations can be seen on patients for Cluster 3:

Cluster 4

In [ ]:
cluster_4 = sample[sample['CLUSTER']==4].value_counts()
cluster_4.to_csv("../results/cluster_4.csv")

print(cluster_4.head(15))

In [ ]:
features = [
    'AGE_GRP','SEX','DIED','L_THREAT', 'ER_VISIT',
    'HOSPITAL','DISABLE','BIRTH_DEFECT','RECOVD', 
    'V_ADMINBY','VAX_MANU','VAX_SITE','VAX_ROUTE'
    ]

xtickslabels = [
    ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110'],
    ['Female', 'Male'],
    ['Lived', 'Died'],
    ['Without Event', 'With Event'],
    ['No ER Visit', 'With ER Visit'],
    ['Not Hospitalized', 'Hospitalized'],
    ['Not Disabled', 'Disabled'],
    ['With No Birth Defect', 'With Birth Defect'],
    ['Recovered', 'Did Not Recover'],
    ['Military','Other',"Pharmacy",'Public','Private','School','Nursing Home','Workplace Clinic'],
    ['JANSSEN', 'MODERNA', 'PFIZER/BIOTECH'],
    ['AR', 'GM', 'LA', 'LG', 'LL', 'OT', 'RA', 'RL'],
    ['Intradermal', 'Intramuscular', 'Needle Free Jet Injector', 'Subcutaneous', 'Needle and Syringe', 'Other']
]

for i in range(len(features)):
    plot_4 = sns.catplot(x=features[i], kind='count', palette='Set2', data=sample.loc[sample['CLUSTER']==4])
    plot_4.axes[0,0].set_xticks([x for x in range(0,len(xtickslabels[i]))])
    plot_4.axes[0,0].set_xticklabels(xtickslabels[i], rotation=90)

The following observations can be seen on patients for Cluster 4:

In [ ]:
cluster_5 = patients[patients['CLUSTER']==5].value_counts()
cluster_5.to_csv("../results/cluster_5.csv")

print(cluster_5.head(15))

In [ ]:
cluster_6 = patients[patients['CLUSTER']==6].value_counts()
cluster_6.to_csv("../results/cluster_6.csv")

print(cluster_6.head(15))

**Identification of Each Cluster**

In [ ]:
cluster_0

## PART 5: Conclusion

**Comparison with Existing Model**

The existing model we chose is the model used in the *Implementation of K-Modes Algorithm for Clustering of Stress Causes in University Students*. Number of clusters was optimized using the **Elbow Method** which is the same method we used.